# Spatial Branching Process Model for Epidemic Diffusion

This is an example implementation of "Epidemic Overdispersion Strengthens the Effectiveness of Mobility Restrictions" (Großmann et al.).
See https://www.researchgate.net/publication/348680143_Epidemic_Overdispersion_Strengthens_the_Effectiveness_of_Mobility_Restrictions for more information.
You can simply run all the cells from top to bottom to get the results.

Note that his is PoC-code and not optimized.
Tested with *Julia 1.5.3*.

In [ ]:
# Only do this once, obviously

#using Pkg
#Pkg.add("StatsBase")  
#Pkg.add("Plots")
#Pkg.add("Distributions")
#Pkg.add("Random")
#Pkg.add("SharedArrays")
#Pkg.add("Distributed")
#Pkg.add("DataFrames")
#Pkg.add("CSV")

# Preliminaries

In [ ]:
using StatsBase
using Plots
using Distributions    
using Random
using SharedArrays
using Distributed
using DataFrames
using CSV
gr()
import Base: floor, ceil

### How to create a model

In [ ]:
mutable struct Model
  agents_num::Int64
  infected_init::Int64
  offspring_dist_type::String
  dispersion::Float64
  r0::Float64
  travel_probability::Float64 
  travel_stop::Union{Float64, Int64}
  rate_func
  sample_point
end

In [ ]:
function rate_func(x::Float64)::Float64
     (1/(x+0.000001))^5
end
sample_point = () -> rand(2)
# example model creation
model=Model(1000, 5, "fixed", NaN, 3.0, 1.0, Inf, rate_func, sample_point)

### Functions


In [ ]:
# re-parametrization of Gamma distribtuion (in epidemiology, we use this formualtion of the Gamma distribution)
function sample_by_k(k::Float64=0.1, r0::Float64=2.0)::Float64
    var::Float64 = r0 ^ 2 / k
    scale::Float64 = var / r0
    g::Gamma = Gamma(k, scale)
    rand(g)
end

# sample number of offsprings
function generate_offspring_count(model::Model)::Int64
    r0::Float64 = model.r0
    if model.offspring_dist_type == "fixed"
        if floor(r0) == ceil(r0)
            return floor(model.r0)
        end
        low = floor(r0)
        return  rand()> r0 - low ? low : low+1
    elseif model.offspring_dist_type == "poisson"
        r0 = model.r0
    elseif model.offspring_dist_type == "gammapoisson"
        r0 = sample_by_k(model.dispersion, model.r0)
    else
        error("unknown distribution type")
    end
    poi::Poisson = Poisson(r0)
    offspring_count::Int64 = rand(poi)
    offspring_count
end

generate_offspring_count(model)

### Code to place the agents (randomly) in the 2D-plane

In [ ]:
function sample_4regions(puffer::Float64=0.1)::Array{Float64,1}
    pos = zeros(2)
    while true
        pos = rand(2)
        if (pos[1] < 0.5-puffer || pos[1] > 0.5+puffer) && (pos[2] < 0.5-puffer || pos[2] > 0.5+puffer)
            break
        end
    end
    pos
end

function sample_16regions()::Array{Float64,1}
    s = 3
    while true
        x1 = sample(0:s)/s
        #x1 = x1 + (rand() -0.5 ) * 0.1
        x1 = x1 + rand(Normal(0.0, 0.03))
        x2 = sample(0:s)/s
        #x2 = x2 + (rand() -0.5 ) * 0.1
        x2 = x2 + rand(Normal(0.0, 0.03))
        if x1 > 0.0  && x1 < 1.0 && x2 > 0.0  && x2 < 1.0 
            return [x1, x2]
        end
    end

end

function sample_16regionsSoft()::Array{Float64,1}
    s = 3
    while true
        x1 = sample(0:s)/s
        #x1 = x1 + (rand() -0.5 ) * 0.1
        x1 = x1 + rand(Normal(0.0, 0.07))
        x2 = sample(0:s)/s
        #x2 = x2 + (rand() -0.5 ) * 0.1
        x2 = x2 + rand(Normal(0.0, 0.07))
        if x1 > 0.0  && x1 < 1.0 && x2 > 0.0  && x2 < 1.0 
            return [x1, x2]
        end
    end

end


function gen_agents(sample_point, agent_num::Int64, puffer::Float64=0.1)::Array{Float64,2}
    agents_pos::Array{Float64,2} = zeros(Float64, (agent_num,2))
    for i=1:size(agents_pos)[1]
        agents_pos[i,:] = sample_point()
    end
    agents_pos
end

gen_agents(sample_4regions, 2)

#sample_structure()

### Simulation Utils

In [ ]:
function get_closest_n(infected_agent::Int64, agents_pos::Array{Float64, 2}, n::Int64)::Set{Int64}
    if n == 0
        return Set([])
    end
    active_pos::Array{Float64,1} = agents_pos[infected_agent,1:2]
    agents::UnitRange{Int64} = 1:size(agents_pos)[1]
    distance::Array{Float64,1} = [sqrt((active_pos[1]-agents_pos[i,1])^2 + (active_pos[2]-agents_pos[i,2])^2) for i in agents]
    distance[infected_agent] = Inf
    distance_i = [(i,distance[i]) for i in agents]
    sort!(distance_i, by=x->x[2])
    my_samps = distance_i[1:n]
    my_samps_i =  [i for (i,_)=my_samps]
    Set(my_samps_i)
end

In [ ]:
function generate_offsprings(infected_agent::Int64, agents_pos::Array{Float64, 2}, offspring_num::Int64, model::Model)::Set{Int64}
    active_pos::Array{Float64,1} = agents_pos[infected_agent,1:2]
    agents::UnitRange{Int64} = 1:size(agents_pos)[1]
    distance::Array{Float64,1} = [sqrt((active_pos[1]-agents_pos[i,1])^2 + (active_pos[2]-agents_pos[i,2])^2) for i in agents]
    weights::Array{Float64,1} = [model.rate_func(d) for d in distance]
    weights[infected_agent] = 0.0
    my_samps::Array{Int64,1} = sample(agents, Weights(weights), offspring_num, replace=false)
    Set(my_samps)
end

In [ ]:
function plot_agents(name::String, agents_pos::Array{Float64, 2}, active_agents::Set{Int64}, recovered_agents::Set{Int64}, step_i)
    plot(agents_pos[:,1], agents_pos[:,2], seriestype = :scatter, title = "Step $(step_i)", color=Colors.JULIA_LOGO_COLORS.blue, legend = false, opacity=0.2)
    aa = collect(active_agents)
    plot!(agents_pos[aa,1], agents_pos[aa,2], seriestype = :scatter, color=Colors.JULIA_LOGO_COLORS.red, legend = false)
    ra = collect(recovered_agents)
    plot!(agents_pos[ra,1], agents_pos[ra,2], seriestype = :scatter, color=Colors.JULIA_LOGO_COLORS.green, legend = false)
    savefig("plots/plot_$(name)_$(step_i+1000).pdf")
end

# Simulation Code

In [ ]:
function simulate(model::Model, plot::Bool = true, name::String="model")::Int64
    agents_pos::Array{Float64, 2} = gen_agents(model.sample_point, model.agents_num)
    recovered_agents::Set{Int64} = Set([])
    active_agents::Set{Int64} = Set([1])
    active_agents = union(active_agents, get_closest_n(1, agents_pos, model.infected_init-1))
    
    if isnan(model.travel_stop) || model.travel_stop < 0 
        model.travel_stop = model.agents_num+1
    end
    
    for step_i = 1:100
        #println(active_agents)
        if step_i == 1 && plot
            plot_agents(name, agents_pos, active_agents, recovered_agents, 0)
        end
        if length(active_agents) == 0
            break
        end
        new_active_agents::Set{Int64} = Set()
        for a_i = active_agents
            if rand() < model.travel_probability && model.travel_stop > length(recovered_agents) # TODO add to model
                agents_pos[a_i,:] = model.sample_point()
            end
            offspring_num::Int64 = generate_offspring_count(model)
            new_active_agents_i::Set{Int64} = generate_offsprings(a_i, agents_pos, offspring_num, model)
            # assert(a_i not in new_active_agents_i)
            new_active_agents = union(new_active_agents, new_active_agents_i)
        end
        new_active_agents = setdiff(new_active_agents, recovered_agents)
        new_active_agents = setdiff(new_active_agents, active_agents)
        recovered_agents = union(recovered_agents, active_agents)  
        active_agents = new_active_agents  
        if plot
            plot_agents(name, agents_pos, active_agents, recovered_agents, step_i) 
        end
    end
    return length(recovered_agents)
end

In [ ]:
function test_run()
    rate_func = x -> (1/(x+0.000001))^2
    
    model=Model(1000, 5, "fixed", NaN, 3.0, 1.0, NaN, rate_func, sample_4regions)
    simulate(model, true, "4regions")

    model=Model(1000, 5, "fixed", NaN, 3.0, 1.0, NaN, rate_func, sample_16regions)
    simulate(model, true, "sample_16regions")
    
    model=Model(1000, 5, "fixed", NaN, 3.0, 1.0, NaN, rate_func, sample_16regionsSoft)
    simulate(model, true, "sample_16regionsSoft")

    model=Model(1000, 5, "fixed", NaN, 3.0, 1.0, NaN, rate_func, () -> rand(2))
    simulate(model, true, "uniform")
end
#test_run()

### Some options for parallelization

In [ ]:
function simulate_n_times(model::Model, sim_num::Int64)::Float64
    mean([simulate(model, false) for i=1:sim_num])
    GC.gc()
end

In [ ]:
function simulate_n_times_loop(model::Model, sim_num::Int64)::Float64
    fds:: Array{Int64,1} = zeros(Int64, sim_num)
    for i=1:sim_num
        begin
            fds[i] = simulate(model, false)
        end
        GC.gc()  # not sure if necessary
    end
    mean(fds)
end

In [ ]:
function simulate_n_times_parallel(model::Model, sim_num::Int64)::Float64
    H=SharedArray(zeros(Int64, sim_num))
    @distributed for i=1:sim_num
        H[i] = simulate(deepcopy(model), false)  #somehow gives all zeros sometimes
    end
    #println(H)  
    mean(H)
end

In [ ]:
# test which parallelization is the fastest

function timing()
    @time begin
      simulate_n_times(model, 100)
    end

    @time begin
      simulate_n_times_loop(model, 100)
    end

    @time begin
      simulate_n_times_parallel(model, 100)
    end
end

# Experiment Code

In [ ]:
function add_row!(df, agents_num, infected_init, offspring_dist_type, dispersion, r0, travel_probability, travel_stop, locality, fds)
    push!(df["agents_num"], agents_num)
    push!(df["infected_init"], infected_init)
    push!(df["offspring_dist_type"], offspring_dist_type)
    push!(df["dispersion"], dispersion)
    push!(df["r0"], r0)
    push!(df["travel_probability"], travel_probability)
    push!(df["travel_stop"], travel_stop)
    push!(df["locality"], locality)
    push!(df["fds"], fds)
end

In [ ]:
function iter_sigma(name::String, steps = 50, sim_num=10000, point_sampler = sample_16regions, travel_stop = Inf, agents_num=1000, minS=0.005, maxS=0.17)
  infected_init::Int64 = 30
  offspring_dist_type::Array{String,1} = ["fixed", "gammapoisson", "poisson"]
  dispersion::Array{Float64,1} = [1.0, 0.5, 0.25, 0.1]
  r0::Float64 = 2.0
  travel_probability::Float64 = 0.0
  df = Dict("agents_num" => [], "infected_init" => [], "offspring_dist_type" => [], "dispersion" => [], "r0" => [], "travel_probability" => [], "travel_stop" => [], "locality" => [], "fds" => [])
  plot_once = true
    
  #locality_factor_range = LinRange(0.005, 0.17, steps)   #for 1000 agents
  locality_factor_range = LinRange(minS, maxS, steps)
  for dispersion_i=dispersion, offspring_dist_type_i=offspring_dist_type
    if offspring_dist_type_i != "gammapoisson" && dispersion_i != dispersion[1]
      continue
    elseif offspring_dist_type_i != "gammapoisson"
      # quick hack to indicate no actual dispersion present
      dispersion_i = -1.0
    end
    fds_range = []
    for lf = locality_factor_range
      rate_func = x -> pdf(Normal(0.0, lf), x)
      model=Model(agents_num, infected_init, offspring_dist_type_i, dispersion_i, r0, travel_probability, travel_stop, rate_func, point_sampler)
      if plot_once
            simulate(model, true, "testPlotSigma_$(name)")
            plot_once = false
      end
      fds = missing 
      try
          fds = simulate_n_times_loop(model, sim_num)
      catch e
          println(e)
      end
      push!(fds_range, fds)
      add_row!(df, agents_num, infected_init, offspring_dist_type_i, dispersion_i, r0, travel_probability, travel_stop, lf, fds)
    end
      plot(locality_factor_range,fds_range, ylims=(0,agents_num))
      savefig("plots/final_$(offspring_dist_type_i)_$(dispersion_i)_$(name).pdf")
  end
  df
end

In [ ]:
function iter_travel(name::String, steps = 50, sim_num=1000, agents_num::Int64 = 1000, point_sampler = sample_16regions, travel_stop = Inf, locality_factor::Float64 = 0.015)
  infected_init::Int64 = 30
  offspring_dist_type::Array{String,1} = ["fixed", "gammapoisson", "poisson"]
  dispersion::Array{Float64,1} = [2.0, 1.0, 0.5, 0.25, 0.1]
  r0::Float64 = 2.0
  #locality_factor::Float64 = 0.015
  df = Dict("agents_num" => [], "infected_init" => [], "offspring_dist_type" => [], "dispersion" => [], "r0" => [], "travel_probability" => [], "travel_stop" => [], "locality" => [], "fds" => [])
  plot_once = true
    
  travel_probability_range = LinRange(0.0, 1.0, steps)
  for dispersion_i=dispersion, offspring_dist_type_i=offspring_dist_type
    if offspring_dist_type_i != "gammapoisson" && dispersion_i != dispersion[1]
      continue
    elseif offspring_dist_type_i != "gammapoisson"
      dispersion_i = -1.0
    end
    fds_range = []
    for tp = travel_probability_range
      rate_func = x -> pdf(Normal(0.0, locality_factor), x)
      model=Model(agents_num, infected_init, offspring_dist_type_i, dispersion_i, r0, tp, travel_stop, rate_func, point_sampler)
      if plot_once
            simulate(model, true, "testPlot_$(name)")
            plot_once = false
      end
      fds = missing 
      try
          fds = simulate_n_times_loop(model, sim_num)
      catch e
          println(e)
      end
      push!(fds_range, fds)
      add_row!(df, agents_num, infected_init, offspring_dist_type_i, dispersion_i, r0, tp, travel_stop, locality_factor, fds)
    end
      plot(travel_probability_range,fds_range, ylims=(0,agents_num))
      savefig("plots/final_$(offspring_dist_type_i)_$(dispersion_i)_$(name).pdf")
  end
  df
end

# Actual Experiments

In [ ]:
# turn these two numbers up to get more accurate results
SIMNUM_10k = 500 
XPOINTS_10k = 30
agent_num = 10000

# create output folder
mkdir("plots")  

### Change Sigma

In [ ]:
name = "ChangeInteractionKernel"
df = iter_sigma(name, XPOINTS_10k, SIMNUM_10k, sample_16regionsSoft, Inf, agent_num, 0.003, 0.05)
df = DataFrame(df)
CSV.write("exp_$(name).csv", df)

### Change Travel Probability

In [ ]:
name = "ChangeTravelProb"
locality_factor_10k = 0.008 #0.07 maybe
df = iter_travel(name, XPOINTS_10k, SIMNUM_10k, agent_num, sample_16regionsSoft, Inf, locality_factor_10k)  #at 0.01
df = DataFrame(df)
CSV.write("exp_$(name)10k.csv", df)

### Change Travel Probability (Delayed)

In [ ]:
name = "ChangeTravelProbAfter200Inf"
locality_factor_10k = 0.007 #0.07 maybe
df = iter_travel(name, XPOINTS_10k, SIMNUM_10k, agent_num, sample_16regionsSoft, 200, locality_factor_10k)
df = DataFrame(df)
CSV.write("exp_$(name).csv", df)

### Fig 1 Visualization

In [ ]:
sigma_small = 0.007
sigma_large = 0.03
sigma_tiny = 0.01 # yeah tiny is smaller than small somehow

rate_func_tinysigma = x -> pdf(Normal(0.0, sigma_tiny), x)
rate_func_smallsigma = x -> pdf(Normal(0.0, sigma_small), x)
rate_func_largesigma = x -> pdf(Normal(0.0, sigma_large), x)

model=Model(10000, 5, "fixed", NaN, 2.0, 0.0, Inf, rate_func_smallsigma, sample_16regionsSoft)
s1 = simulate(model, true, "f1FixedNoTravelSmallSigma")

model=Model(10000, 5, "fixed", NaN, 2.0, 0.0, Inf, rate_func_largesigma, sample_16regionsSoft)
s2 = simulate(model, true, "f1FixedNoTravelLargeSigma")

model=Model(10000, 5, "fixed", NaN, 2.0, 0.1, 200, rate_func_tinysigma, sample_16regionsSoft)
s3 = simulate(model, true, "f1FixedSomeTravelTinySigma")

(s1, s2, s3)
